In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
import os
import numpy as np
import re
import ast
from geopy.distance import geodesic
import json

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
directory = "../../data/processed"

In [4]:
data = pd.read_csv(os.path.join(directory,"categoricals_numericals_raw.csv"))

In [5]:
data

,tour_id,productType,brand,activityLevel,reviewsRating,reviewsTotal,lowestOptionRoomType,lowestOptionPrice,lowestOptionFullPrice,fkSeasonId,tour_option_id,tour_id.1,sourceTourOptionName,isPrivateRequest,maxPax,startLocationName,startLocationCountryCode,startLocationLongitude,startLocationLatitude,endLocationName,endLocationCountryCode,endLocationLongitude,endLocationLatitude,countries_visited,minChildPriceAge,maxChildPriceAge,day_1_accommodation,day_1_meals,day_2_accommodation,day_2_meals,day_3_accommodation,day_3_meals,day_4_accommodation,day_4_meals,day_5_accommodation,day_5_meals,day_6_accommodation,day_6_meals,day_7_accommodation,day_7_meals,day_8_accommodation,day_8_meals,day_9_accommodation,day_9_meals,day_10_accommodation,day_10_meals,day_11_accommodation,day_11_meals,day_12_accommodation,day_12_meals,day_13_accommodation,day_13_meals,day_14_accommodation,day_14_meals,day_15_accommodation,day_15_meals,day_16_accommodation,day_16_meals,day_17_accommodation,day_17_meals,day_18_accommodation,day_18_meals,day_19_accommodation,day_19_meals,day_20_accommodation,day_20_meals,day_21_accommodation,day_21_meals,day_22_accommodation,day_22_meals,day_23_accommodation,day_23_meals,day_24_accommodation,day_24_meals,day_25_accommodation,day_25_meals,day_26_accommodation,day_26_meals,day_27_accommodation,day_27_meals,day_28_accommodation,day_28_meals,locations
0,tour-50422032-fd11-4b38-9334-09b67d61270c,connection_tour,trafalgar,NaN,10.0,2,twin,4405.5,4895.0,06cfb7cd-f9f1-4e90-b9c1-28b6476d7a8a,0044857c-35c9-45f4-97b8-75e7eb7e90f6,tour-50422032-fd11-4b38-9334-09b67d61270c,NaN,0,NaN,Anchorage,NaN,NaN,NaN,Anchorage,NaN,NaN,NaN,United States,5,17,Sheraton Anchorage,Welcome Reception,Denali Park Village,Breakfast,Denali Park Village,Breakfast,River's Edge Resort,"Breakfast, Dinner",River's Edge Resort,"Breakfast, Dinner",Best Western Valdez Harbor Inn,Breakfast,Best Western Valdez Harbor Inn,"Breakfast, Lunch",Sheraton Anchorage,"Breakfast, Dinner",Sheraton Anchorage,"Breakfast, Lunch",NaN,Breakfast,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'name': 'Anchorage', 'countryCode': 'US', 'l..."
1,tour-b75b1cb9-82b6-4626-aa9d-0de564473644,connection_tour,trafalgar,NaN,10.0,2,twin,5017.4,5492.0,8f5363a4-8a86-47ec-a981-3ffe7e3eb5f7,0085cc11-4ee8-4e13-9c07-0e9280376a55,tour-b75b1cb9-82b6-4626-aa9d-0de564473644,NaN,0,NaN,Santiago (Chile),NaN,NaN,NaN,Manaus,NaN,NaN,NaN,"Chile, Argentina, Brazil",5,17,Intercontinental,Welcome Reception,NaN,"Breakfast, Lunch",Cabana del Lago,"Breakfast, Dinner",NH Edelweiss,"Breakfast, Lunch",NaN,Breakfast,Alvear Art,"Breakfast, Be My Guest",NaN,"Breakfast, Regional Dinner",NaN,Breakfast,Bourbon Cataratas Resort Iguassu Falls,"Breakfast, Dinner",NaN,Breakfast,Arena Copacabana,Breakfast,NaN,Breakfast,NaN,"Breakfast, Farewell Dinner",Quality,Breakfast,Amazon Ecopark Jungle Lodge,"Breakfast, Lunch, Dinner",NaN,"Breakfast, Lunch, Dinner",NaN,Breakfast,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'name': 'Rio de Janeiro', 'countryCode': 'BR..."
2,tour-4c051a6b-1146-4690-a6c3-0028da870307,connection_tour,trafalgar,NaN,10.0,2,twin,2472.4,2706.0,2fef3989-d354-457c-a966-90e10293576a,00a948ee-d3f5-4709-a148-b0b06b656227,tour-4c051a6b-1146-4690-a6c3-0028da870307,NaN,0,NaN,Lima,PE,-76.837357,-12.103224,Lima,PE,-76.837357,-12.103224,Peru,5,17,Iberostar Hotel Lima,Welcome Reception,Iberostar Hotel Lima,"Breakfast, Be My Guest",Sonesta Posadas del Inca Yucay,"Breakfast, Lunch, Dinner",El MaPi,"Breakfast, Dinner",Hotel Novotel Cusco,Breakfast,Hotel Novotel Cusco,Breakfast,Iberostar Hotel Lima,"Breakfast, Farewell Dinner",NaN,Breakfast,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'name': 'Cusco', 'countryCode': 'PE', 'longi..."
3,tour-b652f8ee-49ae-476a-b247-f617d4d64d19,c

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 773 entries, 0 to 772
Data columns (total 83 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   tour_id                   773 non-null    object 
 1   productType               773 non-null    object 
 2   brand                     773 non-null    object 
 3   activityLevel             223 non-null    object 
 4   reviewsRating             773 non-null    float64
 5   reviewsTotal              773 non-null    int64  
 6   lowestOptionRoomType      773 non-null    object 
 7   lowestOptionPrice         773 non-null    float64
 8   lowestOptionFullPrice     773 non-null    float64
 9   fkSeasonId                773 non-null    object 
 10  tour_option_id            773 non-null    object 
 11  tour_id.1                 773 non-null    object 
 12  sourceTourOptionName      117 non-null    object 
 13  isPrivateRequest          773 non-null    int64  
 14  maxPax    

In [7]:
drop_list = []

# Meals

In [8]:
meal_columns= ['day_' + str(i) + "_meals" for i in range(1, 29)]

In [9]:
unique_meals = set()

# Iterate through each column that contains meal lists
for col in meal_columns:
    # Split the comma-separated values in each cell and add them to the set
    unique_meals.update(data[col].str.split(',').explode().str.strip())

unique_meals.discard(np.nan)
# Convert the set of unique meal values to a list
unique_meals_list = list(unique_meals)

# Print the list of unique meal values
print(unique_meals_list)

['Breakfast', 'Dinner (one evening only)', 'Dinner', 'Welcome Reception', 'Morning Tea', 'Be My Guest', 'Dinner and local drinks (including wine and spirits)', 'Lunch', 'Regional Dinner', 'Light refreshments on the river cruise', 'Morning tea', 'Cooking Class', 'Kickoff Party', 'Dinner with Wine', 'Farewell Dinner', 'Lunch with Wine', 'Afternoon Tea']


In [10]:
new_meal_columns = []

for meal in unique_meals:
    data[meal] = data.apply(lambda row: any(meal in str(cell) for cell in row[meal_columns]), axis=1).astype(int)
    new_meal_columns.append(meal)
data

,tour_id,productType,brand,activityLevel,reviewsRating,reviewsTotal,lowestOptionRoomType,lowestOptionPrice,lowestOptionFullPrice,fkSeasonId,tour_option_id,tour_id.1,sourceTourOptionName,isPrivateRequest,maxPax,startLocationName,startLocationCountryCode,startLocationLongitude,startLocationLatitude,endLocationName,endLocationCountryCode,endLocationLongitude,endLocationLatitude,countries_visited,minChildPriceAge,maxChildPriceAge,day_1_accommodation,day_1_meals,day_2_accommodation,day_2_meals,day_3_accommodation,day_3_meals,day_4_accommodation,day_4_meals,day_5_accommodation,day_5_meals,day_6_accommodation,day_6_meals,day_7_accommodation,day_7_meals,day_8_accommodation,day_8_meals,day_9_accommodation,day_9_meals,day_10_accommodation,day_10_meals,day_11_accommodation,day_11_meals,day_12_accommodation,day_12_meals,day_13_accommodation,day_13_meals,day_14_accommodation,day_14_meals,day_15_accommodation,day_15_meals,day_16_accommodation,day_16_meals,day_17_accommodation,day_17_meals,day_18_accommodation,day_18_meals,day_19_accommodation,day_19_meals,day_20_accommodation,day_20_meals,day_21_accommodation,day_21_meals,day_22_accommodation,day_22_meals,day_23_accommodation,day_23_meals,day_24_accommodation,day_24_meals,day_25_accommodation,day_25_meals,day_26_accommodation,day_26_meals,day_27_accommodation,day_27_meals,day_28_accommodation,day_28_meals,locations,Breakfast,Dinner (one evening only),Dinner,Welcome Reception,Morning Tea,Be My Guest,Dinner and local drinks (including wine and spirits),Lunch,Regional Dinner,Light refreshments on the river cruise,Morning tea,Cooking Class,Kickoff Party,Dinner with Wine,Farewell Dinner,Lunch with Wine,Afternoon Tea
0,tour-50422032-fd11-4b38-9334-09b67d61270c,connection_tour,trafalgar,NaN,10.0,2,twin,4405.5,4895.0,06cfb7cd-f9f1-4e90-b9c1-28b6476d7a8a,0044857c-35c9-45f4-97b8-75e7eb7e90f6,tour-50422032-fd11-4b38-9334-09b67d61270c,NaN,0,NaN,Anchorage,NaN,NaN,NaN,Anchorage,NaN,NaN,NaN,United States,5,17,Sheraton Anchorage,Welcome Reception,Denali Park Village,Breakfast,Denali Park Village,Breakfast,River's Edge Resort,"Breakfast, Dinner",River's Edge Resort,"Breakfast, Dinner",Best Western Valdez Harbor Inn,Breakfast,Best Western Valdez Harbor Inn,"Breakfast, Lunch",Sheraton Anchorage,"Breakfast, Dinner",Sheraton Anchorage,"Breakfast, Lunch",NaN,Breakfast,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'name': 'Anchorage', 'countryCode': 'US', 'l...",1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0
1,tour-b75b1cb9-82b6-4626-aa9d-0de564473644,connection_tour,trafalgar,NaN,10.0,2,twin,5017.4,5492.0,8f5363a4-8a86-47ec-a981-3ffe7e3eb5f7,0085cc11-4ee8-4e13-9c07-0e9280376a55,tour-b75b1cb9-82b6-4626-aa9d-0de564473644,NaN,0,NaN,Santiago (Chile),NaN,NaN,NaN,Manaus,NaN,NaN,NaN,"Chile, Argentina, Brazil",5,17,Intercontinental,Welcome Reception,NaN,"Breakfast, Lunch",Cabana del Lago,"Breakfast, Dinner",NH Edelweiss,"Breakfast, Lunch",NaN,Breakfast,Alvear Art,"Breakfast, Be My Guest",NaN,"Breakfast, Regional Dinner",NaN,Breakfast,Bourbon Cataratas Resort Iguassu Falls,"Breakfast, Dinner",NaN,Breakfast,Arena Copacabana,Breakfast,NaN,Breakfast,NaN,"Breakfast, Farewell Dinner",Quality,Breakfast,Amazon Ecopark Jungle Lodge,"Breakfast, Lunch, Dinner",NaN,"Breakfast, Lunch, Dinner",NaN,Breakfast,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'name': 'Rio de Janeiro', 'countryCode': 'BR...",1,0,1,1,0,1,0,1,1,0,0,0,0,0,1,0,0
2,tour-4c051a6b-1146-4690-a6c3-0028da870307,connection_tour,trafalgar,NaN,10.0,2,twin,2472.4,2706.0,2fef3989-d354-457c-a966-90e10293576a,00a948ee-d3f5-4709-a148-b0b06b656227,tour-4c051a6b-1146-4690-a6c3-0028da870307,NaN,0,NaN,Lima,PE,-76.837357,-12.103224,Lima,PE,-76.837357,-12.103224,Peru,5,17,Iberostar Hotel Lima,Welcome Reception,Iberostar Hotel Lima,"Breakfast, Be My Guest",Sonesta Posadas del Inca Yucay,"Breakfast, Lunch, Dinner",El MaPi,"Breakfast, Dinner",Hotel Novot

In [11]:
data[new_meal_columns]

,Breakfast,Dinner (one evening only),Dinner,Welcome Reception,Morning Tea,Be My Guest,Dinner and local drinks (including wine and spirits),Lunch,Regional Dinner,Light refreshments on the river cruise,Morning tea,Cooking Class,Kickoff Party,Dinner with Wine,Farewell Dinner,Lunch with Wine,Afternoon Tea
0,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0
1,1,0,1,1,0,1,0,1,1,0,0,0,0,0,1,0,0
2,1,0,1,1,0,1,0,1,0,0,0,0,0,0,1,0,0
3,1,0,1,1,0,1,0,1,0,0,0,0,0,0,1,0,0
4,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
768,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
769,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
770,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
771,1,0,1,1,0,1,0,1,1,0,0,0,0,0,0,0,0


In [12]:
data[new_meal_columns].sum().sort_values(ascending=False)

Breakfast                                               772
Dinner                                                  761
Lunch                                                   483
Dinner with Wine                                        331
Welcome Reception                                       151
Lunch with Wine                                          99
Be My Guest                                              91
Farewell Dinner                                          85
Regional Dinner                                          53
Afternoon Tea                                            25
Kickoff Party                                             9
Morning Tea                                               9
Light refreshments on the river cruise                    1
Morning tea                                               1
Cooking Class                                             1
Dinner (one evening only)                                 1
Dinner and local drinks (including wine 

**High Frequency, Low Uniqueness:** Columns like "Breakfast," "Dinner," and "Lunch" occur very frequently but are not very unique, meaning many tours will have these features, and thus, they may not significantly contribute to differentiating between tours. However, they can still be essential for filtering; for example, if a user specifically wants a tour that includes breakfast.

**Low Frequency, High Uniqueness:** Columns like "Cooking Class," "Morning tea," and "Light refreshments on the river cruise" are unique but occur very infrequently. These can be valuable for differentiation but may apply to very few cases, limiting their overall impact on the recommender.

**Moderate Frequency, Moderate Uniqueness:** Columns like "Dinner with Wine," "Welcome Reception," and "Be My Guest" can be the most beneficial as they offer a balance between uniqueness and frequency, allowing them to significantly contribute to the recommendation logic.


In [13]:
drop_list.extend(meal_columns)

In [14]:
drop_list.extend(['Morning tea', 'Dinner and local drinks (including wine and spirits)', 'Light refreshments on the river cruise', 'Dinner (one evening only)', 'Dinner (one evening only)','Cooking Class' ])
drop_list

['day_1_meals',
 'day_2_meals',
 'day_3_meals',
 'day_4_meals',
 'day_5_meals',
 'day_6_meals',
 'day_7_meals',
 'day_8_meals',
 'day_9_meals',
 'day_10_meals',
 'day_11_meals',
 'day_12_meals',
 'day_13_meals',
 'day_14_meals',
 'day_15_meals',
 'day_16_meals',
 'day_17_meals',
 'day_18_meals',
 'day_19_meals',
 'day_20_meals',
 'day_21_meals',
 'day_22_meals',
 'day_23_meals',
 'day_24_meals',
 'day_25_meals',
 'day_26_meals',
 'day_27_meals',
 'day_28_meals',
 'Morning tea',
 'Dinner and local drinks (including wine and spirits)',
 'Light refreshments on the river cruise',
 'Dinner (one evening only)',
 'Dinner (one evening only)',
 'Cooking Class']

# Accommodation

In [15]:
itineraries = pd.read_csv('../../data/processed/itineraries.csv')

In [16]:
itineraries['accommodation']

0       La Jardin Hotel Haute Couture (Deluxe Classic ...
1                Paradise Sails (Deluxe Cabin) or similar
2       Shining Riverside Hoi An Boutique and Spa (Del...
3       Sensory Park Urban Hotel (Standard Room) or si...
4       La Jardin Hotel Haute Couture (Deluxe Classic ...
                              ...                        
8932                                     Tivoli Innsbruck
8933     Hilton Danube Waterfront, Imperial Riding School
8934                                          Mercure MOA
8935                                 Austria Trend Europa
8936     Hilton Danube Waterfront, Imperial Riding School
Name: accommodation, Length: 8937, dtype: object

In [17]:
itineraries['accommodation'].value_counts()

accommodation
Hotel Astoria, Lucerne.                       99
room 007 Select, Rome.                        95
Le Méridien Etoile, Paris.                    78
Hilton Vienna Park.                           76
Eurostars Residenza Cannaregio, Venice.       72
                                              ..
Ashford Castle                                 1
Starhotels Ritz (Classic Room) or similar      1
Taj Ganges, Varanasi or similar                1
Nikko Princess                                 1
Austria Trend Europa                           1
Name: count, Length: 1725, dtype: int64

In [18]:
itineraries[itineraries['accommodation'] == "Hilton Vienna Park."]['tour_option_id'].value_counts()

tour_option_id
e1d0bf21-2b0a-44c9-8ce3-aabc43ac8d5b    3
b6d71cf4-2881-4304-8229-4cd0a9114c59    2
3829945e-efc8-49a9-b2d2-1ef444741705    2
7f136ca8-d2e9-4d50-b033-94acce3af603    2
fa7e8ff6-0dff-4420-bc4e-98e472dc5f7a    2
7a8b98f3-4d7d-4c2b-9fb9-53a7ba49d65a    2
240ce16c-58bb-442b-a919-8050741ccf4c    2
c22cfbd0-f07c-4812-861e-d2c1629576df    2
23120cbc-0d89-4578-b8b9-b1dc1c46fe47    2
b1b5eb28-9c05-4d1f-b8c3-870df9888d87    2
8042f8c6-cda5-47b7-83f1-e47916054681    2
07d2728e-324d-404c-a1b9-365371aecd1f    2
846e8cdd-2f5a-472d-9d1b-f09f0c998177    2
5e0a6616-347f-4a70-8980-c028e2c19c5f    2
61ea1a8d-dc84-43fd-a15e-014e4e5a957a    2
2be59843-a328-41ce-9916-1948145784b7    2
d86825fc-d858-4795-b78c-b790dfa3b1ab    2
8f023ad7-282a-4ec0-89b5-4077502c2365    2
e8353cc9-72f5-4552-b5a2-16ecf6ec14c6    2
64735c40-19b3-4e08-8158-ebb2a478fae9    2
db14757f-388f-4e83-9cf0-9d97ac797d59    2
2e6e2ef4-cd2a-418e-b19b-a6211dc4ccde    2
27f23fd3-9e2c-4d9d-9ab8-f4786b7925b7    2
901a9adf-0cd0-43ca-

In [19]:
value_counts = itineraries['accommodation'].dropna().value_counts(dropna=False)
(value_counts > 1).sum(), (value_counts > 2).sum(), (value_counts > 3).sum(), (value_counts > 5).sum(), (value_counts > 10).sum(), (value_counts > 15).sum(), (value_counts > 20).sum()

(1375, 864, 698, 391, 153, 82, 48)

In [20]:
# Find values in the 'accommodation' column that occur more than 15 times
new_accommodation_columns = value_counts[value_counts > 15].index.tolist()
new_accommodation_columns

['Hotel Astoria, Lucerne.',
 'room 007 Select, Rome.',
 'Le Méridien Etoile, Paris.',
 'Hilton Vienna Park.',
 'Eurostars Residenza Cannaregio, Venice.',
 'Hotel Adler Cavalieri, Florence.',
 'River Tosca.',
 'AC Hotel Innsbruck.',
 'NH Firenze, Florence.',
 'Hotel Royal Prague.',
 'Grand Mediterraneo',
 'Killarney Plaza Hotel and Spa.',
 'Barceló Costa Vasca, San Sebastián.',
 'Iberostar Hotel Lima',
 'Novotel',
 'Hilton',
 'Barceló Torre de Madrid (Deluxe Room) or similar',
 'Millennium',
 'Crowne Plaza Paris - Republique.',
 'Meliá Sevilla, Seville.',
 'Hotel Novotel Cusco',
 'Crowne Plaza',
 'MV Celestyal Olympia (or similar).',
 'Hilton Barcelona.',
 'MS Medea or MS Miriam (or similar).',
 'Grand Hotel Amrâth Amsterdam.',
 'Hotel Riu Plaza España, Madrid.',
 'Golden Tulip Rome Piram.',
 'Hotel Simplon, Baveno.',
 'Kolbe Hotel Rome.',
 'Divani Caravel, Radisson Blu Park, Crowne Plaza Athens',
 'Outrigger Kona Resort & Spa - Moderate (Mountain View), First Class (Partial Ocean View)

In [21]:
def convert_to_snake_case(input_string):
    # Convert to snake case
    snake_case_string = '_'.join(word.lower() for word in input_string.split())
    
    # Replace characters with '_'
    cleaned_string = re.sub(r'['',.\-()/]', '', snake_case_string)

    # Truncate to a maximum of 40 characters
    if len(cleaned_string) > 40:
        cleaned_string = cleaned_string[:40]
    
    return cleaned_string

In [22]:
new_accommodation_columns_snake = list(map(convert_to_snake_case, new_accommodation_columns))
new_accommodation_columns_snake

['hotel_astoria_lucerne',
 'room_007_select_rome',
 'le_méridien_etoile_paris',
 'hilton_vienna_park',
 'eurostars_residenza_cannaregio_venice',
 'hotel_adler_cavalieri_florence',
 'river_tosca',
 'ac_hotel_innsbruck',
 'nh_firenze_florence',
 'hotel_royal_prague',
 'grand_mediterraneo',
 'killarney_plaza_hotel_and_spa',
 'barceló_costa_vasca_san_sebastián',
 'iberostar_hotel_lima',
 'novotel',
 'hilton',
 'barceló_torre_de_madrid_deluxe_room_or_s',
 'millennium',
 'crowne_plaza_paris__republique',
 'meliá_sevilla_seville',
 'hotel_novotel_cusco',
 'crowne_plaza',
 'mv_celestyal_olympia_or_similar',
 'hilton_barcelona',
 'ms_medea_or_ms_miriam_or_similar',
 'grand_hotel_amrâth_amsterdam',
 'hotel_riu_plaza_españa_madrid',
 'golden_tulip_rome_piram',
 'hotel_simplon_baveno',
 'kolbe_hotel_rome',
 'divani_caravel_radisson_blu_park_crowne_',
 'outrigger_kona_resort_&_spa__moderate_mo',
 'nh_collection_santiago_de_compostela',
 'le_meridien_new_delhi_the_leela_ambience',
 'fairmont_banff_s

In [23]:
accommodation_columns= ['day_' + str(i) + "_accommodation" for i in range(1, 29)]

In [24]:
new_columns_df = pd.DataFrame()

for accommodation in new_accommodation_columns_snake:
    new_columns_df[accommodation] = data.apply(
        lambda row: any(accommodation in convert_to_snake_case(str(cell))
                        for cell in row[accommodation_columns]), axis=1).astype(int)

data = pd.concat([data, new_columns_df], axis=1)


In [25]:
data[new_accommodation_columns_snake]

,hotel_astoria_lucerne,room_007_select_rome,le_méridien_etoile_paris,hilton_vienna_park,eurostars_residenza_cannaregio_venice,hotel_adler_cavalieri_florence,river_tosca,ac_hotel_innsbruck,nh_firenze_florence,hotel_royal_prague,grand_mediterraneo,killarney_plaza_hotel_and_spa,barceló_costa_vasca_san_sebastián,iberostar_hotel_lima,novotel,hilton,barceló_torre_de_madrid_deluxe_room_or_s,millennium,crowne_plaza_paris__republique,meliá_sevilla_seville,hotel_novotel_cusco,crowne_plaza,mv_celestyal_olympia_or_similar,hilton_barcelona,ms_medea_or_ms_miriam_or_similar,grand_hotel_amrâth_amsterdam,hotel_riu_plaza_españa_madrid,golden_tulip_rome_piram,hotel_simplon_baveno,kolbe_hotel_rome,divani_caravel_radisson_blu_park_crowne_,outrigger_kona_resort_&_spa__moderate_mo,nh_collection_santiago_de_compostela,le_meridien_new_delhi_the_leela_ambience,fairmont_banff_springs,amman_marriott_hotel,arcotel_castellani_salzburg,hotel_isabella_sorrento,hilton_garden_inn_budapest,hotel_lev_ljubljana,maritim_hotel_würzburg,grand_hotel_la_chiusa_di_chietri_alberob,munich_marriott_hotel,grand_europe,nh_collection_venezia_murano_villa_venic,hotel_mediterraneo_rome,waterfront_hotel_sliema,dan_panorama,hotel_indigo_venice__sant'elena,celestyal_olympia_exterior_stateroom_or_,aranwa_sacred_valley_hotel_&_wellness,the_twin_fin_waikiki__moderate_city_view,nh_collection_prague_city,occidental_granada_double_room_or_simila,yellowstone_national_park_lodge,sheraton_zagreb_hotel,the_westin_warsaw,shangrila,marriott_hotel_heidelberg,hotel_romanico_palace_rome,evergreen_laurel,divani_caravel_hotel_athens_crowne_plaza,barceló_aran_mantegna,sonesta_posadas_del_inca_yucay,hyatt_regency_maui_resort_and_spa__moder,sofitel_saigon_plaza,divani_palace_acropolis_new_hotel_athens,alvear_art_hotel,vienna_house_andel's_cracow_kraków,radisson_blu_scandinavia_hotel_copenhage,state_game_lodge_custer_state_park,novotel_mestre_castellana,strater_hotel_durango,anuraga_palace,petra_marriott_hotel,meliá_costa_del_sol_torremolinos,vienna_house_by_wyndham_andel's_prague,rg_naxos_hotel_taormina,kimpton_vividora_hotel_barcelona,clayton_hotel_burlington_road_dublin,sheraton_kauai_coconut_beach_resort__mod,cairo_marriott_hotel_&_omar_khayyam_casi
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
768,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
769,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
770,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
771,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [26]:
len(new_accommodation_columns_snake)

82

In [27]:
data[new_accommodation_columns_snake].sum()

hotel_astoria_lucerne                       55
room_007_select_rome                        38
le_méridien_etoile_paris                    46
hilton_vienna_park                          40
eurostars_residenza_cannaregio_venice       36
                                            ..
rg_naxos_hotel_taormina                      8
kimpton_vividora_hotel_barcelona             8
clayton_hotel_burlington_road_dublin         8
sheraton_kauai_coconut_beach_resort__mod     6
cairo_marriott_hotel_&_omar_khayyam_casi     4
Length: 82, dtype: int64

In [28]:
data

,tour_id,productType,brand,activityLevel,reviewsRating,reviewsTotal,lowestOptionRoomType,lowestOptionPrice,lowestOptionFullPrice,fkSeasonId,tour_option_id,tour_id.1,sourceTourOptionName,isPrivateRequest,maxPax,startLocationName,startLocationCountryCode,startLocationLongitude,startLocationLatitude,endLocationName,endLocationCountryCode,endLocationLongitude,endLocationLatitude,countries_visited,minChildPriceAge,maxChildPriceAge,day_1_accommodation,day_1_meals,day_2_accommodation,day_2_meals,day_3_accommodation,day_3_meals,day_4_accommodation,day_4_meals,day_5_accommodation,day_5_meals,day_6_accommodation,day_6_meals,day_7_accommodation,day_7_meals,day_8_accommodation,day_8_meals,day_9_accommodation,day_9_meals,day_10_accommodation,day_10_meals,day_11_accommodation,day_11_meals,day_12_accommodation,day_12_meals,day_13_accommodation,day_13_meals,day_14_accommodation,day_14_meals,day_15_accommodation,day_15_meals,day_16_accommodation,day_16_meals,day_17_accommodation,day_17_meals,day_18_accommodation,day_18_meals,day_19_accommodation,day_19_meals,day_20_accommodation,day_20_meals,day_21_accommodation,day_21_meals,day_22_accommodation,day_22_meals,day_23_accommodation,day_23_meals,day_24_accommodation,day_24_meals,day_25_accommodation,day_25_meals,day_26_accommodation,day_26_meals,day_27_accommodation,day_27_meals,day_28_accommodation,day_28_meals,locations,Breakfast,Dinner (one evening only),Dinner,Welcome Reception,Morning Tea,Be My Guest,Dinner and local drinks (including wine and spirits),Lunch,Regional Dinner,Light refreshments on the river cruise,Morning tea,Cooking Class,Kickoff Party,Dinner with Wine,Farewell Dinner,Lunch with Wine,Afternoon Tea,hotel_astoria_lucerne,room_007_select_rome,le_méridien_etoile_paris,hilton_vienna_park,eurostars_residenza_cannaregio_venice,hotel_adler_cavalieri_florence,river_tosca,ac_hotel_innsbruck,nh_firenze_florence,hotel_royal_prague,grand_mediterraneo,killarney_plaza_hotel_and_spa,barceló_costa_vasca_san_sebastián,iberostar_hotel_lima,novotel,hilton,barceló_torre_de_madrid_deluxe_room_or_s,millennium,crowne_plaza_paris__republique,meliá_sevilla_seville,hotel_novotel_cusco,crowne_plaza,mv_celestyal_olympia_or_similar,hilton_barcelona,ms_medea_or_ms_miriam_or_similar,grand_hotel_amrâth_amsterdam,hotel_riu_plaza_españa_madrid,golden_tulip_rome_piram,hotel_simplon_baveno,kolbe_hotel_rome,divani_caravel_radisson_blu_park_crowne_,outrigger_kona_resort_&_spa__moderate_mo,nh_collection_santiago_de_compostela,le_meridien_new_delhi_the_leela_ambience,fairmont_banff_springs,amman_marriott_hotel,arcotel_castellani_salzburg,hotel_isabella_sorrento,hilton_garden_inn_budapest,hotel_lev_ljubljana,maritim_hotel_würzburg,grand_hotel_la_chiusa_di_chietri_alberob,munich_marriott_hotel,grand_europe,nh_collection_venezia_murano_villa_venic,hotel_mediterraneo_rome,waterfront_hotel_sliema,dan_panorama,hotel_indigo_venice__sant'elena,celestyal_olympia_exterior_stateroom_or_,aranwa_sacred_valley_hotel_&_wellness,the_twin_fin_waikiki__moderate_city_view,nh_collection_prague_city,occidental_granada_double_room_or_simila,yellowstone_national_park_lodge,sheraton_zagreb_hotel,the_westin_warsaw,shangrila,marriott_hotel_heidelberg,hotel_romanico_palace_rome,evergreen_laurel,divani_caravel_hotel_athens_crowne_plaza,barceló_aran_mantegna,sonesta_posadas_del_inca_yucay,hyatt_regency_maui_resort_and_spa__moder,sofitel_saigon_plaza,divani_palace_acropolis_new_hotel_athens,alvear_art_hotel,vienna_house_andel's_cracow_kraków,radisson_blu_scandinavia_hotel_copenhage,state_game_lodge_custer_state_park,novotel_mestre_castellana,strater_hotel_durango,anuraga_palace,petra_marriott_hotel,meliá_costa_del_sol_torremolinos,vienna_house_by_wyndham_andel's_prague,rg_naxos_hotel_taormina,kimpton_vividora_hotel_barcelona,clayton_hotel_burlington_road_dublin,sheraton_kauai_coconut_beach_resort__mod,cairo_marriott_hotel_&_omar_khayyam_casi
0,tour-50422032-fd11-4b38-9334-09b67d61270c,connection_tour,trafalgar,NaN,10.0,2,twin,4405.5,4895.0,06cfb7

In [29]:
drop_list.extend(accommodation_columns)

In [30]:
data = data.drop(drop_list, axis = 1)

# Locations

In [31]:
data['locations'][0]

"[{'name': 'Anchorage', 'countryCode': 'US', 'longitude': -149.60715762529136, 'latitude': 61.10231371193639}, {'name': 'Valdez', 'countryCode': 'US', 'longitude': -145.86079123191877, 'latitude': 61.16852343534351}, {'name': 'Fairbanks', 'countryCode': 'US', 'longitude': -147.22608518037055, 'latitude': 64.92091286788462}, {'name': 'Kenai Fjords National Park', 'countryCode': 'US', 'longitude': -150.0946734616673, 'latitude': 59.827868028897}, {'name': 'Seward', 'countryCode': 'US', 'longitude': -93.23193202690808, 'latitude': 44.9592887256732}, {'name': 'Anchorage', 'countryCode': 'US', 'longitude': -149.60715762529136, 'latitude': 61.10231371193639}, {'name': 'Denali National Park', 'countryCode': 'US', 'longitude': -150.97515094076664, 'latitude': 63.30125320473709}, {'name': 'Valdez', 'countryCode': 'US', 'longitude': -145.86079123191877, 'latitude': 61.16852343534351}, {'name': 'Fairbanks', 'countryCode': 'US', 'longitude': -147.22608518037055, 'latitude': 64.92091286788462}, {'n

## Data Transformation Plan

1. **Number of Locations**
   - **Objective**: Create a new feature representing the number of locations visited during the tour.

2. **Unique Locations**
   - **Objective**: Extract unique locations visited during the tour and represent them as binary features using one-hot encoding.
   - **Example**: Create binary columns like `visited_Anchorage`, `visited_Valdez`, etc.

3. **Country Code**
   - **Objective**: Extract unique country codes visited during the tour and represent them as binary features using one-hot encoding.
   - **Example**: Create binary columns like `visited_country_US`, `visited_country_CA`, etc.

4. **Latitude and Longitude Statistics**
   - **Objective**: Calculate statistics for latitude and longitude across all locations to provide insights into the central tendencies of the tour locations.
   - **Statistics to Calculate**:
     - Mean latitude and longitude
     - Median latitude and longitude
     - Minimum latitude and longitude
     - Maximum latitude and longitude
     - Range of latitude and longitude

5. **Distance Traveled**
   - **Objective**: Calculate the total distance traveled during the tour by summing the distances between consecutive locations.
   - **Method**: Use the Haversine formula to calculate the distance between two latitude-longitude points.

By following this plan, we aim to enhance the understanding of tour data by creating meaningful features and statistics related to the locations visited during the tour.


In [32]:
file_path = '../../data/processed/locations_visited.csv'
locations_visited = pd.read_csv(file_path)

In [33]:
locations_visited

,id,tour_id,itinerary_id,name,countryCode,longitude,latitude,created_at,updated_at
0,1,tour-c6876018-bc87-49e4-9e6b-dbe099ad2a77,1,Hanoi,VN,105.699772,20.998488,2023-09-25 11:50:18,2023-09-25 11:50:18
1,2,tour-c6876018-bc87-49e4-9e6b-dbe099ad2a77,2,Hanoi,VN,105.699772,20.998488,2023-09-25 11:50:18,2023-09-25 11:50:18
2,3,tour-c6876018-bc87-49e4-9e6b-dbe099ad2a77,2,Ha Long Bay,VN,107.051479,20.824062,2023-09-25 11:50:18,2023-09-25 11:50:18
3,4,tour-c6876018-bc87-49e4-9e6b-dbe099ad2a77,3,Hanoi,VN,105.699772,20.998488,2023-09-25 11:50:18,2023-09-25 11:50:18
4,5,tour-c6876018-bc87-49e4-9e6b-dbe099ad2a77,4,Ho Chi Minh City,VN,106.713355,10.743635,2023-09-25 11:50:18,2023-09-25 11:50:18
...,...,...,...,...,...,...,...,...,...
16560,16561,tour-171df543-de65-4733-b863-c7a3cfae7b45,8934,Salzburg,AT,13.046118,47.797631,2023-09-25 11:50:27,2023-09-25 11:50:27
16561,16562,tour-171df543-de65-4733-b863-c7a3cfae7b45,8935,Berlin,DE,13.236281,52.436959,2023-09-25 11:50:27,2023-09-25 11:50:27
16562,16563,tour-171df543-de65-4733-b863-c7a3cfae7b45,8936,Innsbruck,AT,11.380083,47.289626,2023-09-25 11:50:27,2023-09-25 11:50:27
16563,16564,tour-171df543-de65-4733-b863-c7a3cfae7b45,8936,Salzburg,AT,13.046118,47.797631,2023-09-25 11:50:27,2023-09-25 11:50:27


In [34]:
locations_with_tour_option_id = pd.merge(
    locations_visited, 
    itineraries[['id', 'tour_option_id']].rename(columns={'id': 'itinerary_id'}), 
    on='itinerary_id', how='left')
locations_with_tour_option_id

,id,tour_id,itinerary_id,name,countryCode,longitude,latitude,created_at,updated_at,tour_option_id
0,1,tour-c6876018-bc87-49e4-9e6b-dbe099ad2a77,1,Hanoi,VN,105.699772,20.998488,2023-09-25 11:50:18,2023-09-25 11:50:18,b788e686-338d-4ce2-8066-01eabba67a02
1,2,tour-c6876018-bc87-49e4-9e6b-dbe099ad2a77,2,Hanoi,VN,105.699772,20.998488,2023-09-25 11:50:18,2023-09-25 11:50:18,b788e686-338d-4ce2-8066-01eabba67a02
2,3,tour-c6876018-bc87-49e4-9e6b-dbe099ad2a77,2,Ha Long Bay,VN,107.051479,20.824062,2023-09-25 11:50:18,2023-09-25 11:50:18,b788e686-338d-4ce2-8066-01eabba67a02
3,4,tour-c6876018-bc87-49e4-9e6b-dbe099ad2a77,3,Hanoi,VN,105.699772,20.998488,2023-09-25 11:50:18,2023-09-25 11:50:18,b788e686-338d-4ce2-8066-01eabba67a02
4,5,tour-c6876018-bc87-49e4-9e6b-dbe099ad2a77,4,Ho Chi Minh City,VN,106.713355,10.743635,2023-09-25 11:50:18,2023-09-25 11:50:18,b788e686-338d-4ce2-8066-01eabba67a02
...,...,...,...,...,...,...,...,...,...,...
16560,16561,tour-171df543-de65-4733-b863-c7a3cfae7b45,8934,Salzburg,AT,13.046118,47.797631,2023-09-25 11:50:27,2023-09-25 11:50:27,b9d8a3df-b867-4f6d-bea5-8ebd280f5d61
16561,16562,tour-171df543-de65-4733-b863-c7a3cfae7b45,8935,Berlin,DE,13.236281,52.436959,2023-09-25 11:50:27,2023-09-25 11:50:27,b9d8a3df-b867-4f6d-bea5-8ebd280f5d61
16562,16563,tour-171df543-de65-4733-b863-c7a3cfae7b45,8936,Innsbruck,AT,11.380083,47.289626,2023-09-25 11:50:27,2023-09-25 11:50:27,b9d8a3df-b867-4f6d-bea5-8ebd280f5d61
16563,16564,tour-171df543-de65-4733-b863-c7a3cfae7b45,8936,Salzburg,AT,13.046118,47.797631,2023-09-25 11:50:27,2023-09-25 11:50:27,b9d8a3df-b867-4f6d-bea5-8ebd280f5d61


In [35]:
from math import isnan

def calculate_total_distance(locations_df):
    locations = locations_df.to_dict('records')  # Convert DataFrame to a list of dicts
    total_distance = 0
    for i in range(1, len(locations)):
        lat1, lon1 = locations[i-1]['latitude'], locations[i-1]['longitude']
        lat2, lon2 = locations[i]['latitude'], locations[i]['longitude']
        
        if isnan(lat1) or isnan(lon1) or isnan(lat2) or isnan(lon2):
            continue  # Skip if any of the coordinates are NaN
        
        coord1 = (lat1, lon1)
        coord2 = (lat2, lon2)
        total_distance += geodesic(coord1, coord2).miles
    return total_distance

In [36]:
locations_with_tour_option_id.isna().sum()

id                  0
tour_id             0
itinerary_id        0
name                0
countryCode        42
longitude         156
latitude          156
created_at          0
updated_at          0
tour_option_id      0
dtype: int64

In [37]:
locations_with_tour_option_id['countryCode'].fillna('', inplace=True)
locations_with_tour_option_id['longitude'].fillna(0, inplace=True)
locations_with_tour_option_id['latitude'].fillna(0, inplace=True)

In [38]:
# Apply the union_sets function to 'unique_locations' and 'unique_country_codes'
aggregated_data = locations_with_tour_option_id.groupby('tour_option_id').agg({
    'name': [('count', 'count'), ('nunique', 'nunique'), ('location', lambda x: list(set(x)))],
    'countryCode': [('count', 'count'), ('nunique', 'nunique'), ('codes', lambda x: list(set(x)))],
    'latitude': 'mean',
    'longitude': 'mean'
    
}).reset_index()
aggregated_data

tour_option_id  name          \
                                          count nunique   
0    0044857c-35c9-45f4-97b8-75e7eb7e90f6    16       6   
1    0085cc11-4ee8-4e13-9c07-0e9280376a55     8       8   
2    00a948ee-d3f5-4709-a148-b0b06b656227    14       4   
3    00c60fd3-9854-4bc2-a95f-20b9fe86ce97    25      13   
4    01cb2392-0dcf-48b8-807d-9482142a48be    18      10   
..                                    ...   ...     ...   
768  ff076a4f-af6f-4ce8-8cef-1ae5ffeaf2b6    22      13   
769  ff209a35-7165-48d6-a556-aa5660fd93a6    17      10   
770  ffb74ece-5f40-4426-8e6f-1045d6ad5ed6    22      11   
771  ffe8af28-1f4b-4c4e-a6f6-47c2a369f4f2     4       4   
772  fffbe628-ba84-4d9a-99c6-d2c44edf3a52    11       4   

                                                       countryCode          \
                                              location       count nunique   
0    [Seward, Anchorage, Valdez, Denali National Pa...          16       1   
1    [Iguassu Falls (Argentina), Manaus, Amazon Rai...           8       3   
2           [Cusco, Sacred Valley, Lima, Machu Picchu]          14       1   
3    [Santiago (Chile), Mirador Serrano Glacier, Ti...          25       2   
4    [Tel Aviv, Jerusalem, Caesarea, Masada Nationa...          18       1   
..                                                 ...         ...     ...   
768  [Greymouth, Oamaru, Fox Glacier, Gore, Doubtfu...          22       1   
769  [Dublin, Cobh, Galway, Ring of Kerry, Waterfor...          17       1   
770  [Assisi, Umbria, Florence, Rome, Vatican, Moun...          22       2   
771  [Buenos Aires, Iguassu Falls (Brazil), Rio de ...           4       2   
772           [Windsor, London, Cotswolds, Stonehenge]          11       1   

                    latitude   longitude  
            codes       mean        mean  
0            [US]  61.154425 -145.221780  
1    [CL, BR, AR] -25.228071  -53.913502  
2            [PE] -12.926891  -73.791962  
3        [CL, AR] -45.619101  -74.308841  
4            [IL]  32.187071   35.266708  
..            ...        ...         ...  
768          [NZ] -44.375665  169.993527  
769          [IE]  52.559958   -8.273230  
770      [IT, VA]  42.526999   12.719720  
771      [BR, AR] -27.189276  -52.712216  
772          [GB]  51.503667   -0.480551  

[773 rows x 9 columns]

In [39]:
# Flatten the MultiIndex column names
aggregated_data.columns = ['_'.join(col).strip('_') for col in aggregated_data.columns.values]

In [40]:
aggregated_data

,tour_option_id,name_count,name_nunique,name_location,countryCode_count,countryCode_nunique,countryCode_codes,latitude_mean,longitude_mean
0,0044857c-35c9-45f4-97b8-75e7eb7e90f6,16,6,"[Seward, Anchorage, Valdez, Denali National Pa...",16,1,[US],61.154425,-145.221780
1,0085cc11-4ee8-4e13-9c07-0e9280376a55,8,8,"[Iguassu Falls (Argentina), Manaus, Amazon Rai...",8,3,"[CL, BR, AR]",-25.228071,-53.913502
2,00a948ee-d3f5-4709-a148-b0b06b656227,14,4,"[Cusco, Sacred Valley, Lima, Machu Picchu]",14,1,[PE],-12.926891,-73.791962
3,00c60fd3-9854-4bc2-a95f-20b9fe86ce97,25,13,"[Santiago (Chile), Mirador Serrano Glacier, Ti...",25,2,"[CL, AR]",-45.619101,-74.308841
4,01cb2392-0dcf-48b8-807d-9482142a48be,18,10,"[Tel Aviv, Jerusalem, Caesarea, Masada Nationa...",18,1,[IL],32.187071,35.266708
...,...,...,...,...,...,...,...,...,...
768,ff076a4f-af6f-4ce8-8cef-1ae5ffeaf2b6,22,13,"[Greymouth, Oamaru, Fox Glacier, Gore, Doubtfu...",22,1,[NZ],-44.375665,169.993527
769,ff209a35-7165-48d6-a556-aa5660fd93a6,17,10,"[Dublin, Cobh, Galway, Ring of Kerry, Waterfor...",17,1,[IE],52.559958,-8.273230
770,ffb74ece-5f40-4426-8e6f-1045d6ad5ed6,22,11,"[Assisi, Umbria, Florence, Rome, Vatican, Moun...",22,2,"[IT, VA]",42.526999,12.719720
771,ffe8af28-1f4b-4c4e-a6f6-47c2a369f4f2,4,4,"[Buenos Aires, Iguassu Falls (Brazil), Rio de ...",4,2,"[BR, AR]",-27.189276,-52.712216


In [41]:
# Calculate total distance traveled for each tour_option_id
aggregated_data['total_distance_traveled'] = locations_with_tour_option_id.groupby('tour_option_id').apply(calculate_total_distance).reset_index(name='total_distance_traveled')['total_distance_traveled']
aggregated_data

,tour_option_id,name_count,name_nunique,name_location,countryCode_count,countryCode_nunique,countryCode_codes,latitude_mean,longitude_mean,total_distance_traveled
0,0044857c-35c9-45f4-97b8-75e7eb7e90f6,16,6,"[Seward, Anchorage, Valdez, Denali National Pa...",16,1,[US],61.154425,-145.221780,7790.775371
1,0085cc11-4ee8-4e13-9c07-0e9280376a55,8,8,"[Iguassu Falls (Argentina), Manaus, Amazon Rai...",8,3,"[CL, BR, AR]",-25.228071,-53.913502,15075.725890
2,00a948ee-d3f5-4709-a148-b0b06b656227,14,4,"[Cusco, Sacred Valley, Lima, Machu Picchu]",14,1,[PE],-12.926891,-73.791962,3064.282857
3,00c60fd3-9854-4bc2-a95f-20b9fe86ce97,25,13,"[Santiago (Chile), Mirador Serrano Glacier, Ti...",25,2,"[CL, AR]",-45.619101,-74.308841,22831.642689
4,01cb2392-0dcf-48b8-807d-9482142a48be,18,10,"[Tel Aviv, Jerusalem, Caesarea, Masada Nationa...",18,1,[IL],32.187071,35.266708,935.246600
...,...,...,...,...,...,...,...,...,...,...
768,ff076a4f-af6f-4ce8-8cef-1ae5ffeaf2b6,22,13,"[Greymouth, Oamaru, Fox Glacier, Gore, Doubtfu...",22,1,[NZ],-44.375665,169.993527,1804.777015
769,ff209a35-7165-48d6-a556-aa5660fd93a6,17,10,"[Dublin, Cobh, Galway, Ring of Kerry, Waterfor...",17,1,[IE],52.559958,-8.273230,1181.430483
770,ffb74ece-5f40-4426-8e6f-1045d6ad5ed6,22,11,"[Assisi, Umbria, Florence, Rome, Vatican, Moun...",22,2,"[IT, VA]",42.526999,12.719720,2584.021791
771,ffe8af28-1f4b-4c4e-a6f6-47c2a369f4f2,4,4,"[Buenos Aires, Iguassu Falls (Brazil), Rio de ...",4,2,"[BR, AR]",-27.189276,-52.712216,1910.625706


In [42]:
unique_country_codes = locations_with_tour_option_id['countryCode'].unique()
unique_country_codes = np.delete(unique_country_codes, np.where(unique_country_codes == ''))
unique_country_codes

array(['VN', 'KH', 'JP', 'KR', 'IN', 'IT', 'CH', 'ZA', 'ZM', 'ZW', 'BT',
       'NP', 'AU', 'CL', 'RW', 'LK', 'EC', 'TW', 'PE', 'BW', 'ES', 'MA',
       'PT', 'TR', 'MV', 'NO', 'DK', 'CA', 'EG', 'JO', 'TH', 'ID', 'AR',
       'BR', 'MG', 'NZ', 'CN', 'GR', 'CZ', 'HU', 'AT', 'SK', 'KE', 'LI',
       'DE', 'SE', 'US', 'UG', 'FR', 'GB', 'VA', 'MC', 'FI', 'NL', 'BE',
       'SZ', 'LS', 'UY', 'IE', 'PL', 'IL', 'SI', 'HR', 'LU', 'CR', 'RO',
       'MK', 'BG', 'BA', 'AL', 'RS', 'ME', 'GI', 'MY', 'SG', 'TZ', 'AM',
       'GE', 'LA', 'IS', 'LT', 'EE', 'LV', 'MT'], dtype=object)

In [43]:
# Add new columns for each unique country code and set values
for country_code in unique_country_codes:
    # Initialize the column with zeros
    aggregated_data[country_code] = 0
    
    # Set 1 if the country code is in the countryCode_codes column for that row
    for index, row in aggregated_data.iterrows():
        if country_code in row['countryCode_codes']:
            aggregated_data.at[index, country_code] = 1

aggregated_data

,tour_option_id,name_count,name_nunique,name_location,countryCode_count,countryCode_nunique,countryCode_codes,latitude_mean,longitude_mean,total_distance_traveled,VN,KH,JP,KR,IN,IT,CH,ZA,ZM,ZW,BT,NP,AU,CL,RW,LK,EC,TW,PE,BW,ES,MA,PT,TR,MV,NO,DK,CA,EG,JO,TH,ID,AR,BR,MG,NZ,CN,GR,CZ,HU,AT,SK,KE,LI,DE,SE,US,UG,FR,GB,VA,MC,FI,NL,BE,SZ,LS,UY,IE,PL,IL,SI,HR,LU,CR,RO,MK,BG,BA,AL,RS,ME,GI,MY,SG,TZ,AM,GE,LA,IS,LT,EE,LV,MT
0,0044857c-35c9-45f4-97b8-75e7eb7e90f6,16,6,"[Seward, Anchorage, Valdez, Denali National Pa...",16,1,[US],61.154425,-145.221780,7790.775371,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0085cc11-4ee8-4e13-9c07-0e9280376a55,8,8,"[Iguassu Falls (Argentina), Manaus, Amazon Rai...",8,3,"[CL, BR, AR]",-25.228071,-53.913502,15075.725890,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,00a948ee-d3f5-4709-a148-b0b06b656227,14,4,"[Cusco, Sacred Valley, Lima, Machu Picchu]",14,1,[PE],-12.926891,-73.791962,3064.282857,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,00c60fd3-9854-4bc2-a95f-20b9fe86ce97,25,13,"[Santiago (Chile), Mirador Serrano Glacier, Ti...",25,2,"[CL, AR]",-45.619101,-74.308841,22831.642689,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,01cb2392-0dcf-48b8-807d-9482142a48be,18,10,"[Tel Aviv, Jerusalem, Caesarea, Masada Nationa...",18,1,[IL],32.187071,35.266708,935.246600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
768,ff076a4f-af6f-4ce8-8cef-1ae5ffeaf2b6,22,13,"[Greymouth, Oamaru, Fox Glacier, Gore, Doubtfu...",22,1,[NZ],-44.375665,169.993527,1804.777015,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
769,ff209a35-7165-48d6-a556-aa5660fd93a6,17,10,"[Dublin, Cobh, Galway, Ring of Kerry, Waterfor...",17,1,[IE],52.559958,-8.273230,1181.430483,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
770,ffb74ece-5f40-4426-8e6f-1045d6ad5ed6,22,11,"[Assisi, Umbria, Florence, Rome, Vatican, Moun...",22,2,"[IT, VA]",42.526999,12.719720,2584.021791,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
771,ffe8af28-1f4b-4c4e-a6f6-47c2a369f4f2,4,4,"[Buenos Aires, Iguassu Falls (Brazil), Rio de ...",4,2,"[BR, AR]",-27.189276,-52.712216,1910.625706,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [44]:
mean_lat_long = aggregated_data[['latitude_mean', 'longitude_mean']]


In [45]:
# aggregated_data = aggregated_data.drop('countryCode_codes', axis = 1)

In [46]:
name_counts = locations_with_tour_option_id['name'].value_counts()
unique_locations = name_counts[name_counts > 15].index.tolist()
unique_locations

['Rome',
 'Venice',
 'Florence',
 'Paris',
 'Madrid',
 'Lucerne',
 'Vienna',
 'Barcelona',
 'Lima',
 'Prague',
 'London',
 'Athens',
 'Dublin',
 'Seville',
 'Amsterdam',
 'Vatican',
 'Killarney',
 'Cairo',
 'Cusco',
 'Lisbon',
 'Innsbruck',
 'Budapest',
 'Istanbul',
 'Granada',
 'Edinburgh',
 'Delhi',
 'Amman',
 'Sorrento',
 'Jaipur',
 'Ho Chi Minh City',
 'Munich',
 'Sacred Valley',
 'Hanoi',
 'Luxor',
 'Salzburg',
 'Galway',
 'Donostia-San Sebastian',
 'Buenos Aires',
 'Machu Picchu',
 'Hoi An',
 'Dubrovnik',
 'Cappadocia',
 'Valencia',
 'Salamanca',
 'Santiago de Compostela',
 'Rio de Janeiro',
 'Heidelberg',
 'Berlin',
 'Pisa',
 'Yellowstone National Park',
 'Aswan',
 'Ljubljana',
 'Zagreb',
 'Glasgow',
 'Porto',
 'Agra',
 'Kraków',
 'Santiago',
 'Oslo',
 'Banff',
 'York',
 'Assisi',
 'Wurzburg',
 'Warsaw',
 'Bath',
 'Stanserhorn',
 'Avignon',
 'Cape Town',
 'Kathmandu',
 'Petra',
 'Split',
 'Nice',
 'Belfast',
 'Copenhagen',
 'Hue',
 'Santorini',
 'Fes',
 'Cork',
 'Jackson',
 'Wai

In [47]:
len(unique_locations)

255

In [48]:
locations_df = pd.DataFrame(0, index=aggregated_data.index, columns=unique_locations)
locations_df

,Rome,Venice,Florence,Paris,Madrid,Lucerne,Vienna,Barcelona,Lima,Prague,London,Athens,Dublin,Seville,Amsterdam,Vatican,Killarney,Cairo,Cusco,Lisbon,Innsbruck,Budapest,Istanbul,Granada,Edinburgh,Delhi,Amman,Sorrento,Jaipur,Ho Chi Minh City,Munich,Sacred Valley,Hanoi,Luxor,Salzburg,Galway,Donostia-San Sebastian,Buenos Aires,Machu Picchu,Hoi An,Dubrovnik,Cappadocia,Valencia,Salamanca,Santiago de Compostela,Rio de Janeiro,Heidelberg,Berlin,Pisa,Yellowstone National Park,Aswan,Ljubljana,Zagreb,Glasgow,Porto,Agra,Kraków,Santiago,Oslo,Banff,York,Assisi,Wurzburg,Warsaw,Bath,Stanserhorn,Avignon,Cape Town,Kathmandu,Petra,Split,Nice,Belfast,Copenhagen,Hue,Santorini,Fes,Cork,Jackson,Waikiki,Jerusalem,Ring of Kerry,Palermo,Cardiff,Rothenburg ob der Tauber,Izmir,Iguassu Falls,Rapid City,Kaanapali Beach,Boston,Salt Lake City,Quito,Pamukkale,Keauhou,Alberobello,Capri,Jasper National Park of Canada,Queenstown,Waterford,Pompeii,Casablanca,Oviedo,Wadi Rum,Londonderry,Cliffs of Moher,Udaipur,Bangkok,Mykonos,Stockholm,Zürich,Siem Reap,Milan,Auckland,Ranthambore National Park,Colombo,Santander,Taormina,Fátima,Lake Maggiore,Melbourne,Vaduz,Bordeaux,Konya,Helsinki,Limerick,Stratford-upon-Avon,Ha Long Bay,Tokyo,Kalambaka,Baveno,Las Vegas,Marrakesh,Saint Moritz,Dead Sea,Sydney,Zion National Park,Inverness,Puerto Maldonado,Mumbai,Edfu,Perugia,Christchurch,Honolulu,Osaka,Hvar,Geiranger,Çanakkale,Ranthambore,Plymouth,Cinque Terre,Scottish Highlands,Sliema,Évora,Córdoba,Stonehenge,French Riviera,Quebec City,Dresden,Zermatt,Durango,Delphi,Monaco City,Nashville,Kyoto,Kandy,Antalya,Nile River Valley,Bilbao,Lake Powell,Shanghai,Beijing,Bergen,Santa Fe,Kusadasi,Monument Valley,Nafplion,Toronto,Scottsdale,Plitvice Lakes National Park,Konigswinter,New York City,Calgary,Reykjavik,Tel Aviv,Ankara,Victoria Falls,Nuwara Eliya,Kochi,Easter Island,St. Andrews,Montreal,Moab,Glencoe,Rapallo,Neuschwanstein Castle,Chiang Mai,Bryce Canyon National Park,Grand Canyon National Park,Dambulla,San Gimignano,Kapaa,Pamplona,Loire Valley,Kilkenny,Lake Titicaca,Blarney Castle,Albuquerque,Kumarakom,Kahului (Maui),Umbria,Toledo,Costa del Sol,Rotorua,Paracas,Veneto,Montreux,Blarney,Adare,Santa Margherita Ligure,Liverpool,Johannesburg,Torremolinos,Sarajevo,Ottawa,Alta,Grand Teton National Park,Obidos,New Orleans,Cesky Krumlov,Da Nang,Cairns,Icefields Parkway,Catania,Bratislava,Memphis,Vancouver,Santa Cruz Island,Spearfish Canyon,Ollantaytambo,Custer State Park,Frankfurt,Ucross,Olympia,Sankt Goar,Varanasi,Denver,"Washington, D.C.",Ragusa,Ballygalley,Geneva,Cody,Badlands National Park,Crazy Horse Memorial,Crete,Covadonga
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [49]:
locations_df['name_location'] = aggregated_data['name_location']

In [50]:
for location in unique_locations:
    for index, row in locations_df.iterrows():
        if location in row['name_location']:
            locations_df.at[index, location] = 1

locations_df = locations_df.drop('name_location', axis = 1)

In [51]:
aggregated_data = pd.concat([aggregated_data, locations_df], axis=1)


In [52]:
aggregated_data = aggregated_data.drop('name_location', axis = 1)

In [53]:
aggregated_data

,tour_option_id,name_count,name_nunique,countryCode_count,countryCode_nunique,countryCode_codes,latitude_mean,longitude_mean,total_distance_traveled,VN,KH,JP,KR,IN,IT,CH,ZA,ZM,ZW,BT,NP,AU,CL,RW,LK,EC,TW,PE,BW,ES,MA,PT,TR,MV,NO,DK,CA,EG,JO,TH,ID,AR,BR,MG,NZ,CN,GR,CZ,HU,AT,SK,KE,LI,DE,SE,US,UG,FR,GB,VA,MC,FI,NL,BE,SZ,LS,UY,IE,PL,IL,SI,HR,LU,CR,RO,MK,BG,BA,AL,RS,ME,GI,MY,SG,TZ,AM,GE,LA,IS,LT,EE,LV,MT,Rome,Venice,Florence,Paris,Madrid,Lucerne,Vienna,Barcelona,Lima,Prague,London,Athens,Dublin,Seville,Amsterdam,Vatican,Killarney,Cairo,Cusco,Lisbon,Innsbruck,Budapest,Istanbul,Granada,Edinburgh,Delhi,Amman,Sorrento,Jaipur,Ho Chi Minh City,Munich,Sacred Valley,Hanoi,Luxor,Salzburg,Galway,Donostia-San Sebastian,Buenos Aires,Machu Picchu,Hoi An,Dubrovnik,Cappadocia,Valencia,Salamanca,Santiago de Compostela,Rio de Janeiro,Heidelberg,Berlin,Pisa,Yellowstone National Park,Aswan,Ljubljana,Zagreb,Glasgow,Porto,Agra,Kraków,Santiago,Oslo,Banff,York,Assisi,Wurzburg,Warsaw,Bath,Stanserhorn,Avignon,Cape Town,Kathmandu,Petra,Split,Nice,Belfast,Copenhagen,Hue,Santorini,Fes,Cork,Jackson,Waikiki,Jerusalem,Ring of Kerry,Palermo,Cardiff,Rothenburg ob der Tauber,Izmir,Iguassu Falls,Rapid City,Kaanapali Beach,Boston,Salt Lake City,Quito,Pamukkale,Keauhou,Alberobello,Capri,Jasper National Park of Canada,Queenstown,Waterford,Pompeii,Casablanca,Oviedo,Wadi Rum,Londonderry,Cliffs of Moher,Udaipur,Bangkok,Mykonos,Stockholm,Zürich,Siem Reap,Milan,Auckland,Ranthambore National Park,Colombo,Santander,Taormina,Fátima,Lake Maggiore,Melbourne,Vaduz,Bordeaux,Konya,Helsinki,Limerick,Stratford-upon-Avon,Ha Long Bay,Tokyo,Kalambaka,Baveno,Las Vegas,Marrakesh,Saint Moritz,Dead Sea,Sydney,Zion National Park,Inverness,Puerto Maldonado,Mumbai,Edfu,Perugia,Christchurch,Honolulu,Osaka,Hvar,Geiranger,Çanakkale,Ranthambore,Plymouth,Cinque Terre,Scottish Highlands,Sliema,Évora,Córdoba,Stonehenge,French Riviera,Quebec City,Dresden,Zermatt,Durango,Delphi,Monaco City,Nashville,Kyoto,Kandy,Antalya,Nile River Valley,Bilbao,Lake Powell,Shanghai,Beijing,Bergen,Santa Fe,Kusadasi,Monument Valley,Nafplion,Toronto,Scottsdale,Plitvice Lakes National Park,Konigswinter,New York City,Calgary,Reykjavik,Tel Aviv,Ankara,Victoria Falls,Nuwara Eliya,Kochi,Easter Island,St. Andrews,Montreal,Moab,Glencoe,Rapallo,Neuschwanstein Castle,Chiang Mai,Bryce Canyon National Park,Grand Canyon National Park,Dambulla,San Gimignano,Kapaa,Pamplona,Loire Valley,Kilkenny,Lake Titicaca,Blarney Castle,Albuquerque,Kumarakom,Kahului (Maui),Umbria,Toledo,Costa del Sol,Rotorua,Paracas,Veneto,Montreux,Blarney,Adare,Santa Margherita Ligure,Liverpool,Johannesburg,Torremolinos,Sarajevo,Ottawa,Alta,Grand Teton National Park,Obidos,New Orleans,Cesky Krumlov,Da Nang,Cairns,Icefields Parkway,Catania,Bratislava,Memphis,Vancouver,Santa Cruz Island,Spearfish Canyon,Ollantaytambo,Custer State Park,Frankfurt,Ucross,Olympia,Sankt Goar,Varanasi,Denver,"Washington, D.C.",Ragusa,Ballygalley,Geneva,Cody,Badlands National Park,Crazy Horse Memorial,Crete,Covadonga
0,0044857c-35c9-45f4-97b8-75e7eb7e90f6,16,6,16,1,[US],61.154425,-145.221780,7790.775371,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0085cc11-4ee8-4e13-9c07-0e9280376a55,8,8,8,3,"[CL, BR, AR]",-25.228071,-53.913502,15075.725890,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [54]:
data = pd.merge(data, aggregated_data, on='tour_option_id', how='left')

In [55]:
data

,tour_id,productType,brand,activityLevel,reviewsRating,reviewsTotal,lowestOptionRoomType,lowestOptionPrice,lowestOptionFullPrice,fkSeasonId,tour_option_id,tour_id.1,sourceTourOptionName,isPrivateRequest,maxPax,startLocationName,startLocationCountryCode,startLocationLongitude,startLocationLatitude,endLocationName,endLocationCountryCode,endLocationLongitude,endLocationLatitude,countries_visited,minChildPriceAge,maxChildPriceAge,locations,Breakfast,Dinner,Welcome Reception,Morning Tea,Be My Guest,Lunch,Regional Dinner,Kickoff Party,Dinner with Wine,Farewell Dinner,Lunch with Wine,Afternoon Tea,hotel_astoria_lucerne,room_007_select_rome,le_méridien_etoile_paris,hilton_vienna_park,eurostars_residenza_cannaregio_venice,hotel_adler_cavalieri_florence,river_tosca,ac_hotel_innsbruck,nh_firenze_florence,hotel_royal_prague,grand_mediterraneo,killarney_plaza_hotel_and_spa,barceló_costa_vasca_san_sebastián,iberostar_hotel_lima,novotel,hilton,barceló_torre_de_madrid_deluxe_room_or_s,millennium,crowne_plaza_paris__republique,meliá_sevilla_seville,hotel_novotel_cusco,crowne_plaza,mv_celestyal_olympia_or_similar,hilton_barcelona,ms_medea_or_ms_miriam_or_similar,grand_hotel_amrâth_amsterdam,hotel_riu_plaza_españa_madrid,golden_tulip_rome_piram,hotel_simplon_baveno,kolbe_hotel_rome,divani_caravel_radisson_blu_park_crowne_,outrigger_kona_resort_&_spa__moderate_mo,nh_collection_santiago_de_compostela,le_meridien_new_delhi_the_leela_ambience,fairmont_banff_springs,amman_marriott_hotel,arcotel_castellani_salzburg,hotel_isabella_sorrento,hilton_garden_inn_budapest,hotel_lev_ljubljana,maritim_hotel_würzburg,grand_hotel_la_chiusa_di_chietri_alberob,munich_marriott_hotel,grand_europe,nh_collection_venezia_murano_villa_venic,hotel_mediterraneo_rome,waterfront_hotel_sliema,dan_panorama,hotel_indigo_venice__sant'elena,celestyal_olympia_exterior_stateroom_or_,aranwa_sacred_valley_hotel_&_wellness,the_twin_fin_waikiki__moderate_city_view,nh_collection_prague_city,occidental_granada_double_room_or_simila,yellowstone_national_park_lodge,sheraton_zagreb_hotel,the_westin_warsaw,shangrila,marriott_hotel_heidelberg,hotel_romanico_palace_rome,evergreen_laurel,divani_caravel_hotel_athens_crowne_plaza,barceló_aran_mantegna,sonesta_posadas_del_inca_yucay,hyatt_regency_maui_resort_and_spa__moder,sofitel_saigon_plaza,divani_palace_acropolis_new_hotel_athens,alvear_art_hotel,vienna_house_andel's_cracow_kraków,radisson_blu_scandinavia_hotel_copenhage,state_game_lodge_custer_state_park,novotel_mestre_castellana,strater_hotel_durango,anuraga_palace,petra_marriott_hotel,meliá_costa_del_sol_torremolinos,vienna_house_by_wyndham_andel's_prague,rg_naxos_hotel_taormina,kimpton_vividora_hotel_barcelona,clayton_hotel_burlington_road_dublin,sheraton_kauai_coconut_beach_resort__mod,cairo_marriott_hotel_&_omar_khayyam_casi,name_count,name_nunique,countryCode_count,countryCode_nunique,countryCode_codes,latitude_mean,longitude_mean,total_distance_traveled,VN,KH,JP,KR,IN,IT,CH,ZA,ZM,ZW,BT,NP,AU,CL,RW,LK,EC,TW,PE,BW,ES,MA,PT,TR,MV,NO,DK,CA,EG,JO,TH,ID,AR,BR,MG,NZ,CN,GR,CZ,HU,AT,SK,KE,LI,DE,SE,US,UG,FR,GB,VA,MC,FI,NL,BE,SZ,LS,UY,IE,PL,IL,SI,HR,LU,CR,RO,MK,BG,BA,AL,RS,ME,GI,MY,SG,TZ,AM,GE,LA,IS,LT,EE,LV,MT,Rome,Venice,Florence,Paris,Madrid,Lucerne,Vienna,Barcelona,Lima,Prague,London,Athens,Dublin,Seville,Amsterdam,Vatican,Killarney,Cairo,Cusco,Lisbon,Innsbruck,Budapest,Istanbul,Granada,Edinburgh,Delhi,Amman,Sorrento,Jaipur,Ho Chi Minh City,Munich,Sacred Valley,Hanoi,Luxor,Salzburg,Galway,Donostia-San Sebastian,Buenos Aires,Machu Picchu,Hoi An,Dubrovnik,Cappadocia,Valencia,Salamanca,Santiago de Compostela,Rio de Janeiro,Heidelberg,Berlin,Pisa,Yellowstone National Park,Aswan,Ljubljana,Zagreb,Glasgow,Porto,Agra,Kraków,Santiago,Oslo,Banff,York,Assisi,Wurzburg,Warsaw,Bath,Stanserhorn,Avignon,Cape Town,Kathmandu,Petra,Split,Nice,Belfast,Copenhagen,Hue,Santorini,Fes,Cork,Jackson,Waikiki,Jerusalem,Ring of Kerry,Palermo,Cardiff,Rothenburg ob der Tauber,Izmir,Iguassu Falls,Rapid City,Kaanapali Beach,

# NaNs

In [56]:
# Get the count of NaN values in each column
nan_counts = data.isna().sum()

# Filter and print columns that have NaN values with the count
for column, nan_count in nan_counts.items():
    if nan_count > 0:
        print(f"{column}: {nan_count} NaN values")

activityLevel: 550 NaN values
sourceTourOptionName: 656 NaN values
maxPax: 668 NaN values
startLocationCountryCode: 165 NaN values
startLocationLongitude: 163 NaN values
startLocationLatitude: 163 NaN values
endLocationCountryCode: 176 NaN values
endLocationLongitude: 174 NaN values
endLocationLatitude: 174 NaN values


In [57]:
data = data.drop(['sourceTourOptionName', 'maxPax'], axis =1)

In [58]:
data['countryCode_codes']

0              [US]
1      [CL, BR, AR]
2              [PE]
3          [CL, AR]
4              [IL]
           ...     
768            [NZ]
769            [IE]
770        [IT, VA]
771        [BR, AR]
772            [GB]
Name: countryCode_codes, Length: 773, dtype: object

In [59]:
def fillna_with_country_codes(row):
    # If startLocationCountryCode is NaN, fill with the first country in countryCode_codes
    if pd.isna(row['startLocationCountryCode']) and row['countryCode_codes']:
        row['startLocationCountryCode'] = row['countryCode_codes'][0]
    
    # If endLocationCountryCode is NaN, fill with the last country in countryCode_codes
    if pd.isna(row['endLocationCountryCode']) and row['countryCode_codes']:
        row['endLocationCountryCode'] = row['countryCode_codes'][-1]
    
    return row

In [60]:
data = data.apply(fillna_with_country_codes, axis=1)
data

,tour_id,productType,brand,activityLevel,reviewsRating,reviewsTotal,lowestOptionRoomType,lowestOptionPrice,lowestOptionFullPrice,fkSeasonId,tour_option_id,tour_id.1,isPrivateRequest,startLocationName,startLocationCountryCode,startLocationLongitude,startLocationLatitude,endLocationName,endLocationCountryCode,endLocationLongitude,endLocationLatitude,countries_visited,minChildPriceAge,maxChildPriceAge,locations,Breakfast,Dinner,Welcome Reception,Morning Tea,Be My Guest,Lunch,Regional Dinner,Kickoff Party,Dinner with Wine,Farewell Dinner,Lunch with Wine,Afternoon Tea,hotel_astoria_lucerne,room_007_select_rome,le_méridien_etoile_paris,hilton_vienna_park,eurostars_residenza_cannaregio_venice,hotel_adler_cavalieri_florence,river_tosca,ac_hotel_innsbruck,nh_firenze_florence,hotel_royal_prague,grand_mediterraneo,killarney_plaza_hotel_and_spa,barceló_costa_vasca_san_sebastián,iberostar_hotel_lima,novotel,hilton,barceló_torre_de_madrid_deluxe_room_or_s,millennium,crowne_plaza_paris__republique,meliá_sevilla_seville,hotel_novotel_cusco,crowne_plaza,mv_celestyal_olympia_or_similar,hilton_barcelona,ms_medea_or_ms_miriam_or_similar,grand_hotel_amrâth_amsterdam,hotel_riu_plaza_españa_madrid,golden_tulip_rome_piram,hotel_simplon_baveno,kolbe_hotel_rome,divani_caravel_radisson_blu_park_crowne_,outrigger_kona_resort_&_spa__moderate_mo,nh_collection_santiago_de_compostela,le_meridien_new_delhi_the_leela_ambience,fairmont_banff_springs,amman_marriott_hotel,arcotel_castellani_salzburg,hotel_isabella_sorrento,hilton_garden_inn_budapest,hotel_lev_ljubljana,maritim_hotel_würzburg,grand_hotel_la_chiusa_di_chietri_alberob,munich_marriott_hotel,grand_europe,nh_collection_venezia_murano_villa_venic,hotel_mediterraneo_rome,waterfront_hotel_sliema,dan_panorama,hotel_indigo_venice__sant'elena,celestyal_olympia_exterior_stateroom_or_,aranwa_sacred_valley_hotel_&_wellness,the_twin_fin_waikiki__moderate_city_view,nh_collection_prague_city,occidental_granada_double_room_or_simila,yellowstone_national_park_lodge,sheraton_zagreb_hotel,the_westin_warsaw,shangrila,marriott_hotel_heidelberg,hotel_romanico_palace_rome,evergreen_laurel,divani_caravel_hotel_athens_crowne_plaza,barceló_aran_mantegna,sonesta_posadas_del_inca_yucay,hyatt_regency_maui_resort_and_spa__moder,sofitel_saigon_plaza,divani_palace_acropolis_new_hotel_athens,alvear_art_hotel,vienna_house_andel's_cracow_kraków,radisson_blu_scandinavia_hotel_copenhage,state_game_lodge_custer_state_park,novotel_mestre_castellana,strater_hotel_durango,anuraga_palace,petra_marriott_hotel,meliá_costa_del_sol_torremolinos,vienna_house_by_wyndham_andel's_prague,rg_naxos_hotel_taormina,kimpton_vividora_hotel_barcelona,clayton_hotel_burlington_road_dublin,sheraton_kauai_coconut_beach_resort__mod,cairo_marriott_hotel_&_omar_khayyam_casi,name_count,name_nunique,countryCode_count,countryCode_nunique,countryCode_codes,latitude_mean,longitude_mean,total_distance_traveled,VN,KH,JP,KR,IN,IT,CH,ZA,ZM,ZW,BT,NP,AU,CL,RW,LK,EC,TW,PE,BW,ES,MA,PT,TR,MV,NO,DK,CA,EG,JO,TH,ID,AR,BR,MG,NZ,CN,GR,CZ,HU,AT,SK,KE,LI,DE,SE,US,UG,FR,GB,VA,MC,FI,NL,BE,SZ,LS,UY,IE,PL,IL,SI,HR,LU,CR,RO,MK,BG,BA,AL,RS,ME,GI,MY,SG,TZ,AM,GE,LA,IS,LT,EE,LV,MT,Rome,Venice,Florence,Paris,Madrid,Lucerne,Vienna,Barcelona,Lima,Prague,London,Athens,Dublin,Seville,Amsterdam,Vatican,Killarney,Cairo,Cusco,Lisbon,Innsbruck,Budapest,Istanbul,Granada,Edinburgh,Delhi,Amman,Sorrento,Jaipur,Ho Chi Minh City,Munich,Sacred Valley,Hanoi,Luxor,Salzburg,Galway,Donostia-San Sebastian,Buenos Aires,Machu Picchu,Hoi An,Dubrovnik,Cappadocia,Valencia,Salamanca,Santiago de Compostela,Rio de Janeiro,Heidelberg,Berlin,Pisa,Yellowstone National Park,Aswan,Ljubljana,Zagreb,Glasgow,Porto,Agra,Kraków,Santiago,Oslo,Banff,York,Assisi,Wurzburg,Warsaw,Bath,Stanserhorn,Avignon,Cape Town,Kathmandu,Petra,Split,Nice,Belfast,Copenhagen,Hue,Santorini,Fes,Cork,Jackson,Waikiki,Jerusalem,Ring of Kerry,Palermo,Cardiff,Rothenburg ob der Tauber,Izmir,Iguassu Falls,Rapid City,Kaanapali Beach,Boston,Salt Lake City,Quito,

In [61]:
data['startLocationCountryCode'].isna().sum(), data['endLocationCountryCode'].isna().sum()

(0, 0)

In [62]:
def fillna_with_mean(row):
    if pd.isna(row['startLocationLongitude']) and not pd.isna(row['longitude_mean']):
        row['startLocationLongitude'] = row['longitude_mean']
        
    if pd.isna(row['startLocationLatitude']) and not pd.isna(row['latitude_mean']):
        row['startLocationLatitude'] = row['latitude_mean']
        
    if pd.isna(row['endLocationLongitude']) and not pd.isna(row['longitude_mean']):
        row['endLocationLongitude'] = row['longitude_mean']
        
    if pd.isna(row['endLocationLatitude']) and not pd.isna(row['latitude_mean']):
        row['endLocationLatitude'] = row['latitude_mean']
        
    return row

In [63]:
data = data.apply(fillna_with_mean, axis=1)
data['startLocationLongitude'].isna().sum(), data['startLocationLatitude'].isna().sum(), data['endLocationLongitude'].isna().sum(), data['endLocationLatitude'].isna().sum()

(0, 0, 0, 0)

In [64]:
data['activityLevel'].fillna('not_specified', inplace=True)

In [65]:
data.isna().sum().sum()

0

# Encoding

In [66]:
data.select_dtypes(include='object').columns

Index(['tour_id', 'productType', 'brand', 'activityLevel',
       'lowestOptionRoomType', 'fkSeasonId', 'tour_option_id', 'tour_id.1',
       'startLocationName', 'startLocationCountryCode', 'endLocationName',
       'endLocationCountryCode', 'countries_visited', 'locations',
       'countryCode_codes'],
      dtype='object')

In [67]:
tour_ids = data[['tour_option_id', 'tour_id']]

In [68]:
drop_list_2 = ['tour_id', 'tour_option_id', 'fkSeasonId', 'tour_id.1', 'countries_visited', 'locations', 'countryCode_codes', 'isPrivateRequest']

In [69]:
to_one_hot = ['productType', 'brand', 'activityLevel',
       'lowestOptionRoomType', 'startLocationName', 
       'startLocationCountryCode', 'endLocationName',
       'endLocationCountryCode']

In [70]:
data = data.drop(drop_list_2, axis = 1)

In [71]:
data_encoded = pd.get_dummies(data, columns=to_one_hot)

In [72]:
data_encoded

,reviewsRating,reviewsTotal,lowestOptionPrice,lowestOptionFullPrice,startLocationLongitude,startLocationLatitude,endLocationLongitude,endLocationLatitude,minChildPriceAge,maxChildPriceAge,Breakfast,Dinner,Welcome Reception,Morning Tea,Be My Guest,Lunch,Regional Dinner,Kickoff Party,Dinner with Wine,Farewell Dinner,Lunch with Wine,Afternoon Tea,hotel_astoria_lucerne,room_007_select_rome,le_méridien_etoile_paris,hilton_vienna_park,eurostars_residenza_cannaregio_venice,hotel_adler_cavalieri_florence,river_tosca,ac_hotel_innsbruck,nh_firenze_florence,hotel_royal_prague,grand_mediterraneo,killarney_plaza_hotel_and_spa,barceló_costa_vasca_san_sebastián,iberostar_hotel_lima,novotel,hilton,barceló_torre_de_madrid_deluxe_room_or_s,millennium,crowne_plaza_paris__republique,meliá_sevilla_seville,hotel_novotel_cusco,crowne_plaza,mv_celestyal_olympia_or_similar,hilton_barcelona,ms_medea_or_ms_miriam_or_similar,grand_hotel_amrâth_amsterdam,hotel_riu_plaza_españa_madrid,golden_tulip_rome_piram,hotel_simplon_baveno,kolbe_hotel_rome,divani_caravel_radisson_blu_park_crowne_,outrigger_kona_resort_&_spa__moderate_mo,nh_collection_santiago_de_compostela,le_meridien_new_delhi_the_leela_ambience,fairmont_banff_springs,amman_marriott_hotel,arcotel_castellani_salzburg,hotel_isabella_sorrento,hilton_garden_inn_budapest,hotel_lev_ljubljana,maritim_hotel_würzburg,grand_hotel_la_chiusa_di_chietri_alberob,munich_marriott_hotel,grand_europe,nh_collection_venezia_murano_villa_venic,hotel_mediterraneo_rome,waterfront_hotel_sliema,dan_panorama,hotel_indigo_venice__sant'elena,celestyal_olympia_exterior_stateroom_or_,aranwa_sacred_valley_hotel_&_wellness,the_twin_fin_waikiki__moderate_city_view,nh_collection_prague_city,occidental_granada_double_room_or_simila,yellowstone_national_park_lodge,sheraton_zagreb_hotel,the_westin_warsaw,shangrila,marriott_hotel_heidelberg,hotel_romanico_palace_rome,evergreen_laurel,divani_caravel_hotel_athens_crowne_plaza,barceló_aran_mantegna,sonesta_posadas_del_inca_yucay,hyatt_regency_maui_resort_and_spa__moder,sofitel_saigon_plaza,divani_palace_acropolis_new_hotel_athens,alvear_art_hotel,vienna_house_andel's_cracow_kraków,radisson_blu_scandinavia_hotel_copenhage,state_game_lodge_custer_state_park,novotel_mestre_castellana,strater_hotel_durango,anuraga_palace,petra_marriott_hotel,meliá_costa_del_sol_torremolinos,vienna_house_by_wyndham_andel's_prague,rg_naxos_hotel_taormina,kimpton_vividora_hotel_barcelona,clayton_hotel_burlington_road_dublin,sheraton_kauai_coconut_beach_resort__mod,cairo_marriott_hotel_&_omar_khayyam_casi,name_count,name_nunique,countryCode_count,countryCode_nunique,latitude_mean,longitude_mean,total_distance_traveled,VN,KH,JP,KR,IN,IT,CH,ZA,ZM,ZW,BT,NP,AU,CL,RW,LK,EC,TW,PE,BW,ES,MA,PT,TR,MV,NO,DK,CA,EG,JO,TH,ID,AR,BR,MG,NZ,CN,GR,CZ,HU,AT,SK,KE,LI,DE,SE,US,UG,FR,GB,VA,MC,FI,NL,BE,SZ,LS,UY,IE,PL,IL,SI,HR,LU,CR,RO,MK,BG,BA,AL,RS,ME,GI,MY,SG,TZ,AM,GE,LA,IS,LT,EE,LV,MT,Rome,Venice,Florence,Paris,Madrid,Lucerne,Vienna,Barcelona,Lima,Prague,London,Athens,Dublin,Seville,Amsterdam,Vatican,Killarney,Cairo,Cusco,Lisbon,Innsbruck,Budapest,Istanbul,Granada,Edinburgh,Delhi,Amman,Sorrento,Jaipur,Ho Chi Minh City,Munich,Sacred Valley,Hanoi,Luxor,Salzburg,Galway,Donostia-San Sebastian,Buenos Aires,Machu Picchu,Hoi An,Dubrovnik,Cappadocia,Valencia,Salamanca,Santiago de Compostela,Rio de Janeiro,Heidelberg,Berlin,Pisa,Yellowstone National Park,Aswan,Ljubljana,Zagreb,Glasgow,Porto,Agra,Kraków,Santiago,Oslo,Banff,York,Assisi,Wurzburg,Warsaw,Bath,Stanserhorn,Avignon,Cape Town,Kathmandu,Petra,Split,Nice,Belfast,Copenhagen,Hue,Santorini,Fes,Cork,Jackson,Waikiki,Jerusalem,Ring of Kerry,Palermo,Cardiff,Rothenburg ob der Tauber,Izmir,Iguassu Falls,Rapid City,Kaanapali Beach,Boston,Salt Lake City,Quito,Pamukkale,Keauhou,Alberobello,Capri,Jasper National Park of Canada,Queenstown,Waterford,Pompeii,Casablanca,Oviedo,Wadi Rum,Londonderry,Cliffs of Moher,Udaipur,Bangkok,Mykonos,Stockholm,Zürich,Siem Reap,Milan,Auckland,Ranthambore National Park

# Scaling

In [73]:
# Get descriptive statistics for each column
descriptive_stats = data_encoded.describe()

# Identify columns that are not in [0, 1] range
columns_not_in_range = []
for column in descriptive_stats.columns:
    min_value = descriptive_stats.at['min', column]
    max_value = descriptive_stats.at['max', column]
    if min_value < 0 or max_value > 1:
        columns_not_in_range.append(column)


In [74]:
columns_not_in_range

['reviewsRating',
 'reviewsTotal',
 'lowestOptionPrice',
 'lowestOptionFullPrice',
 'startLocationLongitude',
 'startLocationLatitude',
 'endLocationLongitude',
 'endLocationLatitude',
 'minChildPriceAge',
 'maxChildPriceAge',
 'name_count',
 'name_nunique',
 'countryCode_count',
 'countryCode_nunique',
 'latitude_mean',
 'longitude_mean',
 'total_distance_traveled']

In [75]:
scaler = StandardScaler()

# Scale the features not in [0, 1] range
data_encoded[columns_not_in_range] = scaler.fit_transform(data_encoded[columns_not_in_range])

In [76]:
data_encoded

,reviewsRating,reviewsTotal,lowestOptionPrice,lowestOptionFullPrice,startLocationLongitude,startLocationLatitude,endLocationLongitude,endLocationLatitude,minChildPriceAge,maxChildPriceAge,Breakfast,Dinner,Welcome Reception,Morning Tea,Be My Guest,Lunch,Regional Dinner,Kickoff Party,Dinner with Wine,Farewell Dinner,Lunch with Wine,Afternoon Tea,hotel_astoria_lucerne,room_007_select_rome,le_méridien_etoile_paris,hilton_vienna_park,eurostars_residenza_cannaregio_venice,hotel_adler_cavalieri_florence,river_tosca,ac_hotel_innsbruck,nh_firenze_florence,hotel_royal_prague,grand_mediterraneo,killarney_plaza_hotel_and_spa,barceló_costa_vasca_san_sebastián,iberostar_hotel_lima,novotel,hilton,barceló_torre_de_madrid_deluxe_room_or_s,millennium,crowne_plaza_paris__republique,meliá_sevilla_seville,hotel_novotel_cusco,crowne_plaza,mv_celestyal_olympia_or_similar,hilton_barcelona,ms_medea_or_ms_miriam_or_similar,grand_hotel_amrâth_amsterdam,hotel_riu_plaza_españa_madrid,golden_tulip_rome_piram,hotel_simplon_baveno,kolbe_hotel_rome,divani_caravel_radisson_blu_park_crowne_,outrigger_kona_resort_&_spa__moderate_mo,nh_collection_santiago_de_compostela,le_meridien_new_delhi_the_leela_ambience,fairmont_banff_springs,amman_marriott_hotel,arcotel_castellani_salzburg,hotel_isabella_sorrento,hilton_garden_inn_budapest,hotel_lev_ljubljana,maritim_hotel_würzburg,grand_hotel_la_chiusa_di_chietri_alberob,munich_marriott_hotel,grand_europe,nh_collection_venezia_murano_villa_venic,hotel_mediterraneo_rome,waterfront_hotel_sliema,dan_panorama,hotel_indigo_venice__sant'elena,celestyal_olympia_exterior_stateroom_or_,aranwa_sacred_valley_hotel_&_wellness,the_twin_fin_waikiki__moderate_city_view,nh_collection_prague_city,occidental_granada_double_room_or_simila,yellowstone_national_park_lodge,sheraton_zagreb_hotel,the_westin_warsaw,shangrila,marriott_hotel_heidelberg,hotel_romanico_palace_rome,evergreen_laurel,divani_caravel_hotel_athens_crowne_plaza,barceló_aran_mantegna,sonesta_posadas_del_inca_yucay,hyatt_regency_maui_resort_and_spa__moder,sofitel_saigon_plaza,divani_palace_acropolis_new_hotel_athens,alvear_art_hotel,vienna_house_andel's_cracow_kraków,radisson_blu_scandinavia_hotel_copenhage,state_game_lodge_custer_state_park,novotel_mestre_castellana,strater_hotel_durango,anuraga_palace,petra_marriott_hotel,meliá_costa_del_sol_torremolinos,vienna_house_by_wyndham_andel's_prague,rg_naxos_hotel_taormina,kimpton_vividora_hotel_barcelona,clayton_hotel_burlington_road_dublin,sheraton_kauai_coconut_beach_resort__mod,cairo_marriott_hotel_&_omar_khayyam_casi,name_count,name_nunique,countryCode_count,countryCode_nunique,latitude_mean,longitude_mean,total_distance_traveled,VN,KH,JP,KR,IN,IT,CH,ZA,ZM,ZW,BT,NP,AU,CL,RW,LK,EC,TW,PE,BW,ES,MA,PT,TR,MV,NO,DK,CA,EG,JO,TH,ID,AR,BR,MG,NZ,CN,GR,CZ,HU,AT,SK,KE,LI,DE,SE,US,UG,FR,GB,VA,MC,FI,NL,BE,SZ,LS,UY,IE,PL,IL,SI,HR,LU,CR,RO,MK,BG,BA,AL,RS,ME,GI,MY,SG,TZ,AM,GE,LA,IS,LT,EE,LV,MT,Rome,Venice,Florence,Paris,Madrid,Lucerne,Vienna,Barcelona,Lima,Prague,London,Athens,Dublin,Seville,Amsterdam,Vatican,Killarney,Cairo,Cusco,Lisbon,Innsbruck,Budapest,Istanbul,Granada,Edinburgh,Delhi,Amman,Sorrento,Jaipur,Ho Chi Minh City,Munich,Sacred Valley,Hanoi,Luxor,Salzburg,Galway,Donostia-San Sebastian,Buenos Aires,Machu Picchu,Hoi An,Dubrovnik,Cappadocia,Valencia,Salamanca,Santiago de Compostela,Rio de Janeiro,Heidelberg,Berlin,Pisa,Yellowstone National Park,Aswan,Ljubljana,Zagreb,Glasgow,Porto,Agra,Kraków,Santiago,Oslo,Banff,York,Assisi,Wurzburg,Warsaw,Bath,Stanserhorn,Avignon,Cape Town,Kathmandu,Petra,Split,Nice,Belfast,Copenhagen,Hue,Santorini,Fes,Cork,Jackson,Waikiki,Jerusalem,Ring of Kerry,Palermo,Cardiff,Rothenburg ob der Tauber,Izmir,Iguassu Falls,Rapid City,Kaanapali Beach,Boston,Salt Lake City,Quito,Pamukkale,Keauhou,Alberobello,Capri,Jasper National Park of Canada,Queenstown,Waterford,Pompeii,Casablanca,Oviedo,Wadi Rum,Londonderry,Cliffs of Moher,Udaipur,Bangkok,Mykonos,Stockholm,Zürich,Siem Reap,Milan,Auckland,Ranthambore National Park

In [77]:
def convert_to_snake_case(name):
    # Replace all non-alphanumeric characters with underscores
    s = re.sub('[\W_]+', '_', name)
    # Convert to lowercase
    return s.lower()

In [78]:
new_columns = [convert_to_snake_case(col) for col in data_encoded.columns]
data_encoded.columns = new_columns
data_encoded

,reviewsrating,reviewstotal,lowestoptionprice,lowestoptionfullprice,startlocationlongitude,startlocationlatitude,endlocationlongitude,endlocationlatitude,minchildpriceage,maxchildpriceage,breakfast,dinner,welcome_reception,morning_tea,be_my_guest,lunch,regional_dinner,kickoff_party,dinner_with_wine,farewell_dinner,lunch_with_wine,afternoon_tea,hotel_astoria_lucerne,room_007_select_rome,le_méridien_etoile_paris,hilton_vienna_park,eurostars_residenza_cannaregio_venice,hotel_adler_cavalieri_florence,river_tosca,ac_hotel_innsbruck,nh_firenze_florence,hotel_royal_prague,grand_mediterraneo,killarney_plaza_hotel_and_spa,barceló_costa_vasca_san_sebastián,iberostar_hotel_lima,novotel,hilton,barceló_torre_de_madrid_deluxe_room_or_s,millennium,crowne_plaza_paris_republique,meliá_sevilla_seville,hotel_novotel_cusco,crowne_plaza,mv_celestyal_olympia_or_similar,hilton_barcelona,ms_medea_or_ms_miriam_or_similar,grand_hotel_amrâth_amsterdam,hotel_riu_plaza_españa_madrid,golden_tulip_rome_piram,hotel_simplon_baveno,kolbe_hotel_rome,divani_caravel_radisson_blu_park_crowne_,outrigger_kona_resort_spa_moderate_mo,nh_collection_santiago_de_compostela,le_meridien_new_delhi_the_leela_ambience,fairmont_banff_springs,amman_marriott_hotel,arcotel_castellani_salzburg,hotel_isabella_sorrento,hilton_garden_inn_budapest,hotel_lev_ljubljana,maritim_hotel_würzburg,grand_hotel_la_chiusa_di_chietri_alberob,munich_marriott_hotel,grand_europe,nh_collection_venezia_murano_villa_venic,hotel_mediterraneo_rome,waterfront_hotel_sliema,dan_panorama,hotel_indigo_venice_sant_elena,celestyal_olympia_exterior_stateroom_or_,aranwa_sacred_valley_hotel_wellness,the_twin_fin_waikiki_moderate_city_view,nh_collection_prague_city,occidental_granada_double_room_or_simila,yellowstone_national_park_lodge,sheraton_zagreb_hotel,the_westin_warsaw,shangrila,marriott_hotel_heidelberg,hotel_romanico_palace_rome,evergreen_laurel,divani_caravel_hotel_athens_crowne_plaza,barceló_aran_mantegna,sonesta_posadas_del_inca_yucay,hyatt_regency_maui_resort_and_spa_moder,sofitel_saigon_plaza,divani_palace_acropolis_new_hotel_athens,alvear_art_hotel,vienna_house_andel_s_cracow_kraków,radisson_blu_scandinavia_hotel_copenhage,state_game_lodge_custer_state_park,novotel_mestre_castellana,strater_hotel_durango,anuraga_palace,petra_marriott_hotel,meliá_costa_del_sol_torremolinos,vienna_house_by_wyndham_andel_s_prague,rg_naxos_hotel_taormina,kimpton_vividora_hotel_barcelona,clayton_hotel_burlington_road_dublin,sheraton_kauai_coconut_beach_resort_mod,cairo_marriott_hotel_omar_khayyam_casi,name_count,name_nunique,countrycode_count,countrycode_nunique,latitude_mean,longitude_mean,total_distance_traveled,vn,kh,jp,kr,in,it,ch,za,zm,zw,bt,np,au,cl,rw,lk,ec,tw,pe,bw,es,ma,pt,tr,mv,no,dk,ca,eg,jo,th,id,ar,br,mg,nz,cn,gr,cz,hu,at,sk,ke,li,de,se,us,ug,fr,gb,va,mc,fi,nl,be,sz,ls,uy,ie,pl,il,si,hr,lu,cr,ro,mk,bg,ba,al,rs,me,gi,my,sg,tz,am,ge,la,is,lt,ee,lv,mt,rome,venice,florence,paris,madrid,lucerne,vienna,barcelona,lima,prague,london,athens,dublin,seville,amsterdam,vatican,killarney,cairo,cusco,lisbon,innsbruck,budapest,istanbul,granada,edinburgh,delhi,amman,sorrento,jaipur,ho_chi_minh_city,munich,sacred_valley,hanoi,luxor,salzburg,galway,donostia_san_sebastian,buenos_aires,machu_picchu,hoi_an,dubrovnik,cappadocia,valencia,salamanca,santiago_de_compostela,rio_de_janeiro,heidelberg,berlin,pisa,yellowstone_national_park,aswan,ljubljana,zagreb,glasgow,porto,agra,kraków,santiago,oslo,banff,york,assisi,wurzburg,warsaw,bath,stanserhorn,avignon,cape_town,kathmandu,petra,split,nice,belfast,copenhagen,hue,santorini,fes,cork,jackson,waikiki,jerusalem,ring_of_kerry,palermo,cardiff,rothenburg_ob_der_tauber,izmir,iguassu_falls,rapid_city,kaanapali_beach,boston,salt_lake_city,quito,pamukkale,keauhou,alberobello,capri,jasper_national_park_of_canada,queenstown,waterford,pompeii,casablanca,oviedo,wadi_rum,londonderry,cliffs_of_moher,udaipur,bangkok,mykonos,stockholm,zürich,siem_reap,milan,auckland,ranthambore_national_park,colombo,san

In [79]:
file_path = os.path.join(directory, f"categoricals_numericals_encoded_scaled.csv")
data_encoded.to_csv(file_path, index=False)

In [80]:
file_path = os.path.join(directory, f"reference_tour_ids.csv")
tour_ids.to_csv(file_path, index=False)

In [81]:
file_path = os.path.join(directory, f"mean_lat_long.csv")
mean_lat_long.to_csv(file_path, index=False)